In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd
from datetime import datetime

Since there is basically no external data file, this notebook is extremly simple to open in collab and to train your models there. This is highly recommended since LSTMS take longer to train. 

# **Task 3: **

**We're going to build a network that takes and converts dates from one format into another. **

For example, given a date string such as "14-03-2020", we want out network to, character by character read this string and output to us "The 14th of March 2020".

Since our data is a sequence of information, each part derives it's meaning from a prior part.
"2" as the second month character could either encode for Feb or for december depending on what number preceded it. This is a problem that is well handled by recurrent neural networks. 

We're going to be using LSTM's to build this network, which are recurrent learning cells. 


Below is a model that allows us to do sequence to sequence conversion where the input and output are of different lengths, the example provided is one of english to french translation. This is similar to the encoder, decoder style of machine translation we have learnt about in class.


![alt text](https://blog.keras.io/img/seq2seq/seq2seq-teacher-forcing.png)






Below is a function that generates the dataset, giving you date entries in different formats in for as many days (2019 April 15th onwards) as you'd like.
Go ahead, test it, see how it returns values and what they are.

### Make Dataset

In [ ]:
def make_short_date(dt):
    return dt.strftime('%d-%m-%Y')

def make_long_date(dt):
    date = dt.strftime('%d')
    if date[-1] == '1':
        suffix = 'st'
    elif date[-1] == '2':
        suffix = 'nd'
    elif date[-1] == '3':
        suffix = 'rd'
    else:
        suffix = 'th'
    month = dt.strftime('%B')
    year = dt.strftime('%Y')
    
    return date + suffix + ' of ' + month + ' ' + year

def make_dataset(n):
    dates = pd.date_range(datetime(1900, 4, 14), periods=n, normalize=True)
    
    x = dates.map(make_short_date).values
    y = dates.map(make_long_date).values
    
    return x, y

In [ ]:
x, y = make_dataset(50)
x[:5], y[:5]

(array(['14-04-1990', '15-04-1990', '16-04-1990', '17-04-1990',
        '18-04-1990'], dtype=object),
 array(['14th of April 1990', '15th of April 1990', '16th of April 1990',
        '17th of April 1990', '18th of April 1990'], dtype=object))

We've got some hyper-paramters set for you here, we're going to start working with 10,000 training examples and see how well our models trains with that.

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

In [ ]:
dataset = make_dataset(num_samples)

In [ ]:
dataset[0][1]

'15-04-1990'

In [ ]:
dataset[1][1]

'15th of April 1990'

### Part 1 - Generation and preperation of dataset

Prepare the dataset for training. The following steps will have to be taken.

We need a total of 3 datasets: 
1. encoder_input (our original data)
2. decoder_input (the target data with start and end tokens added) -> our start token is a "\t" character, and the stop character "\n".
3. decoder_target (target data without a start token, but with an end token) 

decoder_input and decoder_target data are different since once the model is trained, we will pass the decoder a sequence containing only a "\t" and it will generate the rest of the sentence for us after, ending with the "\n" token.

Here is an example of this format of data for a single sample.

encoder_input: "14-03-2019"
decoder_input: "\tThe 14th of March 2019\n"
decoder_target: "The 14th of March 2019\n"

Now that we know what the target is for the dataset, it's time to start converting it into a form the network can understand and work with.
We need each sample to be an n*m numpy array of 0's. Where n is the maximum length of the sequence and m is the vocubulary size.

An input would go from "14-05-19" to a array of size (1*8*10), where 1 is our batch size, 8 is sequence length and our vocab is 10 (including the '-').

To do this, complete the following:

1. Create a list of all possible vocab for the input and output target data (use a set)
2. Use this set to create a dictionary that can convert characters into ints

    *For instance you'll have a 'char_2_index' array that will function as "char_2_index['-'] = 13"*
3. Convert these lists of ints into a 2d numpy array (3d when considering batches)

**Example: **

Input sentence: 14-04-2019 into a 3d tensor would result in the following:


[[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]

  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]]
-


In [ ]:
# code help: https://keras.io/examples/nlp/lstm_seq2seq/
# create list to contain input dates and target format of dates
input_dates = []
target_dates = []
# create set (as mentioned in instructions) of individual characters
input_chars = set()
target_chars = set()

for i in range(len(dataset[0])):

    # populate input_dates list
    input_date = dataset[0][i]
    input_dates.append(input_date)

    # populate target_dates list
    target_date = "\t" + dataset[1][i] + "\n"
    target_dates.append(target_date)
    
    # populate input_chars set
    for char in input_date:
        if char not in input_chars:
            input_chars.add(char)

    # populate target_chars set
    for char in target_date:
        if char not in target_chars:
            target_chars.add(char)

# make set to list and sort
input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))

max_len_encoder_seq = max([len(txt) for txt in input_dates])
max_len_decoder_seq = max([len(txt) for txt in target_dates])

input_token_index = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_index = dict([(char, i) for i, char in enumerate(target_chars)])

encoder_input = np.zeros( (len(input_dates), max_len_encoder_seq, len(input_chars)))
decoder_input = np.zeros( (len(input_dates), max_len_decoder_seq, len(target_chars)))
decoder_target = np.zeros( (len(input_dates), max_len_decoder_seq, len(target_chars)))

for batch in len(input_dates):
    for t, char in enumerate(input_dates[batch]):
        encoder_input[batch, t, input_token_index[char]] = 1.0
    
    for t, char in enumerate(target_dates[batch]):
        # decoder_target is ahead of decoder_input by one timestep
        decoder_input[batch, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target[batch, t - 1, target_token_index[char]] = 1.0
    decoder_input[batch, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target[batch, t:, target_token_index[" "]] = 1.0

**Part 2 - Setting up the network**

Before we begin, uncomment the following lines of code and fill in appropriate variables to have an overview of what your network will be training with.

In [ ]:
print('Number of samples:', num_samples)
print('Number of unique input tokens:', len(input_chars))
print('Number of unique output tokens:', len(target_chars))
print('Max sequence length for inputs:', max_len_encoder_seq)
print('Max sequence length for outputs:', max_len_decoder_seq)

Number of samples: 10000
Number of unique input tokens: 11
Number of unique output tokens: 40
Max sequence length for inputs: 10
Max sequence length for outputs: 24


Great, now you have to set up an encoder decoder network. 

This will require 2 LSTMS

1. An encoder LSTM (size - latent dimension as we defined above):
  - We'll pass our encoder_input data to this
  - We will let it run through the LSTM and get the states back from it (discard the network output, we only need the c and h states), save these
 
2. A decoder LSTM (size - latent dimension):
  - We'll be passing decoder_input data to this (with the '\t' and ''\n' added and encoded)
  - We will also be passing a specific initial state to this (states c and h, taken from the encoder network)
  
Following this LSTM, you will need a dense layer of output_tokens (output vocab) size to convert the result into a one hot encoded target. Figure out what activation this should require

In [ ]:
encoder_inputs = Input(shape=(None, len(input_chars)))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

In [ ]:
# decoder
decoder_inputs = Input(shape=(None, len(target_chars)))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(len(target_chars), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# Model definition
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

** Model structure ** 

So you have 

  1. (encoder_input) -> encoder LSTM -> (output, states)
  2. (decoder_input, states) -> decoder LSTM -> Dense (decoder_output)
  
For the overall model: 
1. Inputs - [encoder_input, decoder_input]
2. Outputs - [decoder_target]

Model Optimizer - RMSProp
Model Loss - categorical_crossentropy


In [ ]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit([encoder_input, decoder_input], decoder_target, batch_size=batch_size, epochs=epochs, validation_split=0.2,)

Epoch 1/100
125/125 [==============================] - 27s 189ms/step - loss: 1.8721 - accuracy: 0.4826 - val_loss: 1.2375 - val_accuracy: 0.6105
Epoch 2/100
125/125 [==============================] - 26s 211ms/step - loss: 0.9051 - accuracy: 0.7305 - val_loss: 0.6767 - val_accuracy: 0.7596
Epoch 3/100
125/125 [==============================] - 23s 182ms/step - loss: 0.5057 - accuracy: 0.8226 - val_loss: 0.4950 - val_accuracy: 0.7925
Epoch 4/100
125/125 [==============================] - 26s 210ms/step - loss: 0.3731 - accuracy: 0.8566 - val_loss: 0.4047 - val_accuracy: 0.8431
Epoch 5/100
125/125 [==============================] - 31s 247ms/step - loss: 0.3190 - accuracy: 0.8741 - val_loss: 0.3371 - val_accuracy: 0.8675
Epoch 6/100
125/125 [==============================] - 26s 204ms/step - loss: 0.2709 - accuracy: 0.8888 - val_loss: 0.3722 - val_accuracy: 0.8326
Epoch 7/100
125/125 [==============================] - 22s 178ms/step - loss: 0.2502 - accuracy: 0.8930 - val_loss: 0.3738 -

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# save the model
path = "/content/drive/MyDrive/Deep Learning/A6/saved_model/"
model.save(path + "PA6_task3")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Deep Learning/A6/saved_model/PA6_task3/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Deep Learning/A6/saved_model/PA6_task3/assets


In [ ]:
# load model for future use
path = "/content/drive/MyDrive/Deep Learning/A6/saved_model/"
model = models.load_model("PA6_task3")

** Generating results **

Now that you've trained the network, you need to create two smaller subnetworks so that you can use them indepedantly for predictions:

1. An encoder model to give you (encoder_input) -> (model states)
2. a decoder model to give you (model_states + start_token) -> (next character)

You will have to use these as following: 

  1. encode input and retrieve initial decoder state
  
  2. run one step of decoder with this initial state and a "start of sequence" token as target.
  
  Output will be the next target token
  
  3. Repeat with the current target token and current states

The following illustration should help solidify this prediction loop better. 



![alt text](https://blog.keras.io/img/seq2seq/seq2seq-inference.png)

In [ ]:
# Define sampling models

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, len(target_chars)))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if sampled_char == "\n" or len(decoded_sentence) > max_len_decoder_seq:
            stop_condition = True
        target_seq = np.zeros((1, 1, len(target_chars)))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence


In [ ]:
# generate 5 random model outputs from inputs
for random in range(5):
    input_seq = encoder_input[random : random + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("Input date:", input_dates[random])
    print("Decoded date in other format:", decoded_sentence)

Input date: 14-04-1990
Decoded date in other format: 14th of April 1990

Input date: 15-04-1990
Decoded date in other format: 15th of April 1990

Input date: 16-04-1990
Decoded date in other format: 16th of April 1990

Input date: 17-04-1990
Decoded date in other format: 17th of April 1990

Input date: 18-04-1990
Decoded date in other format: 18th of April 1990



In [ ]:
# date from 1987
input_date_test = "18-04-1987"

encoder_input_test = np.zeros( ( 1, max_len_encoder_seq, len(input_chars)) )
print(encoder_input_test.shape)

for t, char in enumerate(input_date_test):
    encoder_input_test[0, t, input_token_index[char]] = 1.0

decoded_sentence = decode_sequence(encoder_input_test)
print("Input date:", input_date_test)
print("Decoded date in other format:", decoded_sentence)

(1, 10, 11)
Input date: 18-04-1987
Decoded date in other format: 18th of April 1998



In [ ]:
# date from 2034
input_date_test = "18-04-2034"

encoder_input_test = np.zeros( ( 1, max_len_encoder_seq, len(input_chars)) )
print(encoder_input_test.shape)

for t, char in enumerate(input_date_test):
    encoder_input_test[0, t, input_token_index[char]] = 1.0

decoded_sentence = decode_sequence(encoder_input_test)
print("Input date:", input_date_test)
print("Decoded date in other format:", decoded_sentence)

(1, 10, 11)
Input date: 18-04-2034
Decoded date in other format: 18th of April 2004



In [ ]:
# date from 2134
input_date_test = "18-04-2134"

encoder_input_test = np.zeros( ( 1, max_len_encoder_seq, len(input_chars)) )
print(encoder_input_test.shape)

for t, char in enumerate(input_date_test):
    encoder_input_test[0, t, input_token_index[char]] = 1.0

decoded_sentence = decode_sequence(encoder_input_test)
print("Input date:", input_date_test)
print("Decoded date in other format:", decoded_sentence)

(1, 10, 11)
Input date: 18-04-2134
Decoded date in other format: 18th of April 2004



In [ ]:
datetime(1990, 4, 14)

datetime.datetime(1990, 4, 14, 0, 0)

** Part 3 - Improving result ** 

Now that you've got a working model, answer the following questions. 

1. What does the model return for a date from 1987? Why?
2. What about a date from 2034?
3. Now try the same date but in year 2134, what does the model return? Why is this so?
4. How do we fix this problem?


Answers:

1. Model returns same date from 1998
2. Model returns same date from 2004
3. Model returns same date from 2004
4. In make dataset() function, dates = pd.date_range(datetime(1990, 4, 14), periods=n, normalize=True) generates the dates. datetime(1990,4,14) gives the start. So start date is 14-04-1990. when n = 10000, dataset was created such that end date was 10,000 days ahead ie 29-08-2017. Model has not seen dates beyond 29-08-2017 and before 14-04-1990. Essentially, model has not seen dates before year 1990 and after year 2017. In order so solve this problem, we can create a more inclusive dataset that covers a greater range of dates.



In [ ]:
# improve the 'generate dataset' function to overcome the limitations you've highlighted in the previous part, use your answer to (4) for this
# code this function below

In [ ]:
def make_dataset(n):
    dates = pd.date_range(datetime(1900, 4, 14), periods=n, freq="W", normalize=True)
    
    x = dates.map(make_short_date).values
    y = dates.map(make_long_date).values
    
    return x, y

In [ ]:
x,y = make_dataset(10000) 

In [ ]:
x

array(['15-04-1900', '22-04-1900', '29-04-1900', ..., '18-11-2091',
       '25-11-2091', '02-12-2091'], dtype=object)

What did you change in this new version of the function?

How will it help improve model results for the specific data points we mentioned earlier that our model had trouble with?

In [ ]:
# as can be seen from the data array of x, the new dataset has dates stretching 
# from 1990 till 2091.

In [ ]:
# This improvement has been achieved by using the freq option in pd.date_range
# weekley frequency has been set by using "W" option.

### Retrain the model

Generate Data Function

In [ ]:
def make_short_date(dt):
    return dt.strftime('%d-%m-%Y')

def make_long_date(dt):
    date = dt.strftime('%d')
    if date[-1] == '1':
        suffix = 'st'
    elif date[-1] == '2':
        suffix = 'nd'
    elif date[-1] == '3':
        suffix = 'rd'
    else:
        suffix = 'th'
    month = dt.strftime('%B')
    year = dt.strftime('%Y')
    
    return date + suffix + ' of ' + month + ' ' + year

def make_dataset(n):
    dates = pd.date_range(datetime(1900, 4, 14), periods=n, freq="W", normalize=True)
    
    x = dates.map(make_short_date).values
    y = dates.map(make_long_date).values
    
    return x, y

Create Dataset

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 40  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 15000  # Number of samples to train on.
dataset = make_dataset(num_samples)

In [ ]:
dataset

(array(['15-04-1900', '22-04-1900', '29-04-1900', ..., '16-09-2187',
        '23-09-2187', '30-09-2187'], dtype=object),
 array(['15th of April 1900', '22th of April 1900', '29th of April 1900',
        ..., '16th of September 2187', '23th of September 2187',
        '30th of September 2187'], dtype=object))

BOW character implementarion data representation

In [ ]:
# code help: https://keras.io/examples/nlp/lstm_seq2seq/
# create list to contain input dates and target format of dates
input_dates = []
target_dates = []
# create set (as mentioned in instructions) of individual characters
input_chars = set()
target_chars = set()

for i in range(len(dataset[0])):

    # populate input_dates list
    input_date = dataset[0][i]
    input_dates.append(input_date)

    # populate target_dates list
    target_date = "\t" + dataset[1][i] + "\n"
    target_dates.append(target_date)
    
    # populate input_chars set
    for char in input_date:
        if char not in input_chars:
            input_chars.add(char)

    # populate target_chars set
    for char in target_date:
        if char not in target_chars:
            target_chars.add(char)

# make set to list and sort
input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))

max_len_encoder_seq = max([len(txt) for txt in input_dates])
max_len_decoder_seq = max([len(txt) for txt in target_dates])

input_token_index = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_index = dict([(char, i) for i, char in enumerate(target_chars)])

encoder_input = np.zeros( (len(input_dates), max_len_encoder_seq, len(input_chars)))
decoder_input = np.zeros( (len(input_dates), max_len_decoder_seq, len(target_chars)))
decoder_target = np.zeros( (len(input_dates), max_len_decoder_seq, len(target_chars)))

for batch in range(len(input_dates)):
    for t, char in enumerate(input_dates[batch]):
        encoder_input[batch, t, input_token_index[char]] = 1.0
    
    for t, char in enumerate(target_dates[batch]):
        # decoder_target is ahead of decoder_input by one timestep
        decoder_input[batch, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target[batch, t - 1, target_token_index[char]] = 1.0
    decoder_input[batch, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target[batch, t:, target_token_index[" "]] = 1.0

Model Definition

In [ ]:
encoder_inputs = Input(shape=(None, len(input_chars)))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

In [ ]:
# decoder
decoder_inputs = Input(shape=(None, len(target_chars)))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(len(target_chars), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# Model definition
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit([encoder_input, decoder_input], decoder_target, batch_size=batch_size, epochs=epochs, validation_split=0.2,)

Epoch 1/40
188/188 [==============================] - 41s 187ms/step - loss: 0.4999 - accuracy: 0.8091 - val_loss: 0.4986 - val_accuracy: 0.7985
Epoch 2/40
188/188 [==============================] - 34s 182ms/step - loss: 0.4019 - accuracy: 0.8380 - val_loss: 0.4626 - val_accuracy: 0.8321
Epoch 3/40
188/188 [==============================] - 35s 184ms/step - loss: 0.3455 - accuracy: 0.8554 - val_loss: 0.4891 - val_accuracy: 0.8340
Epoch 4/40
188/188 [==============================] - 35s 184ms/step - loss: 0.3251 - accuracy: 0.8599 - val_loss: 0.4610 - val_accuracy: 0.8319
Epoch 5/40
188/188 [==============================] - 35s 187ms/step - loss: 0.3003 - accuracy: 0.8705 - val_loss: 0.4489 - val_accuracy: 0.8128
Epoch 6/40
188/188 [==============================] - 36s 189ms/step - loss: 0.2623 - accuracy: 0.8924 - val_loss: 0.4385 - val_accuracy: 0.8517
Epoch 7/40
188/188 [==============================] - 36s 191ms/step - loss: 0.2268 - accuracy: 0.9123 - val_loss: 0.4688 - val_ac

Evaluate model on unknown data:

In [ ]:
# Define sampling models

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, len(target_chars)))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if sampled_char == "\n" or len(decoded_sentence) > max_len_decoder_seq:
            stop_condition = True
        target_seq = np.zeros((1, 1, len(target_chars)))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence


In [ ]:
input_date_test = ["10-12-1987", "18-04-2134", "05-04-2034"] 

for i, input_date in enumerate(input_date_test):
    encoder_input_test = np.zeros( ( 1, max_len_encoder_seq, len(input_chars)) )

    for t, char in enumerate(input_date):
        encoder_input_test[0, t, input_token_index[char]] = 1.0

    decoded_sentence = decode_sequence(encoder_input_test)
    print("Input date:", input_date)
    print("Decoded date in other format:", decoded_sentence)

Input date: 10-12-1987
Decoded date in other format: 10th of December 1987

Input date: 18-04-2134
Decoded date in other format: 18th of April 2134

Input date: 05-04-2034
Decoded date in other format: 05th of April 2034



### As seen from the output, there is no apparent problem with the retrained model. 1987, 2134, and 2034 dates have been accurately converted from one date-format to the other.